# Day 2: 데이터 정제 및 구조 파악 (Data Cleaning)

**날짜**: 2025-07-05

**목표**:
- Day 1에서 생성한 샘플 데이터 로드
- 자치구명 표준화 및 일치 여부 재확인
- CCTV 유형별 집계 및 분석
- 범죄 유형별 집계 및 분석
- 이상치 탐지 및 처리 방안 결정
- 정제된 데이터 저장

**활용 도구**:
- utils 모듈의 공통 함수 적극 활용
- 데이터 품질 검증

In [ ]:
# 라이브러리 임포트
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from utils.constants import (
    SEOUL_DISTRICTS, CCTV_RANGE, CRIME_RANGE, CCTV_EFFECT_CRIMES,
    RANDOM_SEED, DATA_PATHS, ANALYSIS_YEAR
)
from utils.helpers import (
    set_korean_font, set_plot_style, print_data_info, save_csv_safely,
    validate_data, create_summary_stats, check_district_consistency
)

warnings.filterwarnings('ignore')

# 환경 설정
set_korean_font()
set_plot_style()
pd.set_option('display.max_columns', None)

print("\n✅ 모든 라이브러리 임포트 완료")
print(f"📊 분석 연도: {ANALYSIS_YEAR}")
print(f"🎯 Day 2: 데이터 정제 및 구조 파악\n")

## 1. Day 1에서 생성한 데이터 로드

In [ ]:
# 데이터 파일 경로
cctv_path = os.path.join(DATA_PATHS['raw'], 'cctv_seoul_2023_sample.csv')
crime_path = os.path.join(DATA_PATHS['raw'], 'crime_seoul_2023_sample.csv')
population_path = os.path.join(DATA_PATHS['raw'], 'population_seoul_2023_sample.csv')

# 데이터 로드
try:
    cctv_df = pd.read_csv(cctv_path, encoding='utf-8-sig')
    crime_df = pd.read_csv(crime_path, encoding='utf-8-sig')
    population_df = pd.read_csv(population_path, encoding='utf-8-sig')
    print("✅ 모든 데이터 파일 로드 성공\n")
except FileNotFoundError as e:
    print(f"❌ 파일을 찾을 수 없습니다: {e}")
    print("⚠️ Day 1 노트북을 먼저 실행하세요.")

# 데이터 정보 출력
print_data_info(cctv_df, "CCTV")
print_data_info(crime_df, "범죄")
print_data_info(population_df, "인구")

## 2. 자치구명 표준화 및 일치 여부 확인

In [ ]:
# 자치구명 표준화 함수
def standardize_district_name(name):
    """
    자치구명 표준화
    - 앞뒤 공백 제거
    - 특수문자 제거
    - '서울특별시', '서울시' 등 접두사 제거
    """
    name = str(name).strip()
    name = name.replace('서울특별시 ', '').replace('서울시 ', '').replace('서울 ', '')
    return name

# 각 데이터프레임의 자치구명 표준화
cctv_df['자치구'] = cctv_df['자치구'].apply(standardize_district_name)
crime_df['자치구'] = crime_df['자치구'].apply(standardize_district_name)
population_df['자치구'] = population_df['자치구'].apply(standardize_district_name)

print("✅ 자치구명 표준화 완료\n")

In [ ]:
# 자치구명 일치 여부 확인
consistency_result = check_district_consistency(
    cctv_df, crime_df, population_df,
    district_col='자치구'
)

if consistency_result['consistent']:
    print("\n✅ 모든 데이터의 자치구명이 일치합니다.")
    print(f"   공통 자치구 수: {len(consistency_result['common_districts'])}개")
else:
    print("\n⚠️ 자치구명 불일치 발견. 매핑 테이블 작성 필요.")
    print(f"   CCTV에만 있는 자치구: {consistency_result['district_sets'][0] - consistency_result['common_districts']}")
    print(f"   범죄에만 있는 자치구: {consistency_result['district_sets'][1] - consistency_result['common_districts']}")
    print(f"   인구에만 있는 자치구: {consistency_result['district_sets'][2] - consistency_result['common_districts']}")

## 3. CCTV 데이터 정제 및 집계

In [ ]:
# CCTV 유형별 기초 통계
print("📊 CCTV 유형별 기초 통계:\n")
cctv_stats = create_summary_stats(
    cctv_df,
    list(CCTV_RANGE.keys()) + ['총_CCTV']
)
display(cctv_stats)

# CCTV 유형별 비율 계산
cctv_df['방범용_비율'] = (cctv_df['방범용'] / cctv_df['총_CCTV'] * 100).round(2)
cctv_df['교통단속용_비율'] = (cctv_df['교통단속용'] / cctv_df['총_CCTV'] * 100).round(2)
cctv_df['어린이안전용_비율'] = (cctv_df['어린이안전용'] / cctv_df['총_CCTV'] * 100).round(2)

print("\n✅ CCTV 유형별 비율 계산 완료")

In [ ]:
# CCTV 유형별 비율 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 좌측: 유형별 평균 대수
cctv_types = list(CCTV_RANGE.keys())
cctv_means = [cctv_df[cctv_type].mean() for cctv_type in cctv_types]

axes[0].bar(cctv_types, cctv_means, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'],
            edgecolor='black', alpha=0.7)
axes[0].set_title('CCTV 유형별 평균 대수', fontsize=14, fontweight='bold')
axes[0].set_xlabel('CCTV 유형', fontsize=12)
axes[0].set_ylabel('평균 대수', fontsize=12)
axes[0].grid(axis='y', alpha=0.3)

# 각 막대 위에 값 표시
for i, (ctype, mean_val) in enumerate(zip(cctv_types, cctv_means)):
    axes[0].text(i, mean_val + 30, f'{mean_val:.0f}', ha='center', fontsize=10, fontweight='bold')

# 우측: 유형별 비율 (파이 차트)
cctv_ratio_means = [
    cctv_df['방범용_비율'].mean(),
    cctv_df['교통단속용_비율'].mean(),
    cctv_df['어린이안전용_비율'].mean(),
    100 - cctv_df['방범용_비율'].mean() - cctv_df['교통단속용_비율'].mean() - cctv_df['어린이안전용_비율'].mean()
]

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
axes[1].pie(cctv_ratio_means, labels=cctv_types, autopct='%1.1f%%',
            colors=colors, startangle=90, textprops={'fontsize': 11})
axes[1].set_title('CCTV 유형별 평균 비율', fontsize=14, fontweight='bold')

plt.tight_layout()
save_path = os.path.join(DATA_PATHS['figures'], 'day2_cctv_types_analysis.png')
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"✅ 그래프 저장: {save_path}")
plt.show()

print("\n📊 CCTV 유형별 비율 분석:")
print(f"   방범용 평균 비율: {cctv_df['방범용_비율'].mean():.2f}%")
print(f"   교통단속용 평균 비율: {cctv_df['교통단속용_비율'].mean():.2f}%")
print(f"   어린이안전용 평균 비율: {cctv_df['어린이안전용_비율'].mean():.2f}%")

In [ ]:
# CCTV 상위/하위 5개 자치구
print("🏆 CCTV 대수 상위 5개 자치구:\n")
top5_cctv = cctv_df.nlargest(5, '총_CCTV')[['자치구', '총_CCTV', '방범용', '교통단속용']]
display(top5_cctv)

print("\n📉 CCTV 대수 하위 5개 자치구:\n")
bottom5_cctv = cctv_df.nsmallest(5, '총_CCTV')[['자치구', '총_CCTV', '방범용', '교통단속용']]
display(bottom5_cctv)

## 4. 범죄 데이터 정제 및 집계

In [ ]:
# 범죄 유형별 기초 통계
print("📊 범죄 유형별 기초 통계:\n")
crime_stats = create_summary_stats(
    crime_df,
    list(CRIME_RANGE.keys()) + ['CCTV효과범죄_합계', '총_범죄']
)
display(crime_stats)

# 범죄 유형별 비율 계산
for crime_type in CRIME_RANGE.keys():
    crime_df[f'{crime_type}_비율'] = (crime_df[crime_type] / crime_df['총_범죄'] * 100).round(2)

crime_df['CCTV효과범죄_비율'] = (crime_df['CCTV효과범죄_합계'] / crime_df['총_범죄'] * 100).round(2)

print("\n✅ 범죄 유형별 비율 계산 완료")

In [ ]:
# 범죄 유형별 비율 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 좌측: 유형별 평균 건수
crime_types = list(CRIME_RANGE.keys())
crime_means = [crime_df[crime_type].mean() for crime_type in crime_types]

axes[0].bar(crime_types, crime_means,
            color=['#d62728', '#ff7f0e', '#2ca02c', '#1f77b4', '#9467bd'],
            edgecolor='black', alpha=0.7)
axes[0].set_title('범죄 유형별 평균 건수', fontsize=14, fontweight='bold')
axes[0].set_xlabel('범죄 유형', fontsize=12)
axes[0].set_ylabel('평균 건수', fontsize=12)
axes[0].grid(axis='y', alpha=0.3)
axes[0].tick_params(axis='x', rotation=15)

# 각 막대 위에 값 표시
for i, (ctype, mean_val) in enumerate(zip(crime_types, crime_means)):
    axes[0].text(i, mean_val + 10, f'{mean_val:.0f}', ha='center', fontsize=10, fontweight='bold')

# 우측: CCTV 효과 범죄 vs 전체 범죄
cctv_effect_ratio = crime_df['CCTV효과범죄_비율'].mean()
other_ratio = 100 - cctv_effect_ratio

axes[1].pie([cctv_effect_ratio, other_ratio],
            labels=['CCTV효과범죄', '기타범죄'],
            autopct='%1.1f%%',
            colors=['#d62728', '#9467bd'],
            startangle=90,
            textprops={'fontsize': 12})
axes[1].set_title('CCTV 효과 범죄 비율', fontsize=14, fontweight='bold')

plt.tight_layout()
save_path = os.path.join(DATA_PATHS['figures'], 'day2_crime_types_analysis.png')
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"✅ 그래프 저장: {save_path}")
plt.show()

print(f"\n📊 CCTV 효과 범죄 비율: {cctv_effect_ratio:.2f}%")
print(f"   절도 평균: {crime_df['절도'].mean():.0f}건")
print(f"   강도 평균: {crime_df['강도'].mean():.0f}건")
print(f"   차량범죄 평균: {crime_df['차량범죄'].mean():.0f}건")

In [ ]:
# 범죄 상위/하위 5개 자치구
print("🏆 범죄 건수 상위 5개 자치구 (CCTV 효과 범죄 기준):\n")
top5_crime = crime_df.nlargest(5, 'CCTV효과범죄_합계')[['자치구', 'CCTV효과범죄_합계', '절도', '강도', '차량범죄']]
display(top5_crime)

print("\n📉 범죄 건수 하위 5개 자치구 (CCTV 효과 범죄 기준):\n")
bottom5_crime = crime_df.nsmallest(5, 'CCTV효과범죄_합계')[['자치구', 'CCTV효과범죄_합계', '절도', '강도', '차량범죄']]
display(bottom5_crime)

## 5. 인구 데이터 정제

In [ ]:
# 인구 데이터 기초 통계
print("📊 인구 데이터 기초 통계:\n")
pop_stats = create_summary_stats(population_df, ['인구수', '면적_km2', '인구밀도'])
display(pop_stats)

# 인구 상위/하위 5개 자치구
print("\n🏆 인구수 상위 5개 자치구:\n")
top5_pop = population_df.nlargest(5, '인구수')[['자치구', '인구수', '면적_km2', '인구밀도']]
display(top5_pop)

print("\n📉 인구수 하위 5개 자치구:\n")
bottom5_pop = population_df.nsmallest(5, '인구수')[['자치구', '인구수', '면적_km2', '인구밀도']]
display(bottom5_pop)

## 6. 이상치 탐지 및 처리 방안 결정

In [ ]:
# IQR 방법을 이용한 이상치 탐지 함수
def detect_outliers_iqr(df, column):
    """
    IQR (Interquartile Range) 방법으로 이상치 탐지
    Q1 - 1.5*IQR 미만 또는 Q3 + 1.5*IQR 초과 값을 이상치로 판정
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]

    return outliers, lower_bound, upper_bound

# CCTV 이상치 탐지
print("🔍 CCTV 이상치 탐지 (IQR 방법):\n")
cctv_outliers, cctv_lower, cctv_upper = detect_outliers_iqr(cctv_df, '총_CCTV')
print(f"   정상 범위: {cctv_lower:.0f} ~ {cctv_upper:.0f}")
print(f"   이상치 개수: {len(cctv_outliers)}개")
if len(cctv_outliers) > 0:
    print(f"   이상치 자치구:")
    display(cctv_outliers[['자치구', '총_CCTV']])

# 범죄 이상치 탐지
print("\n🔍 범죄 이상치 탐지 (IQR 방법):\n")
crime_outliers, crime_lower, crime_upper = detect_outliers_iqr(crime_df, 'CCTV효과범죄_합계')
print(f"   정상 범위: {crime_lower:.0f} ~ {crime_upper:.0f}")
print(f"   이상치 개수: {len(crime_outliers)}개")
if len(crime_outliers) > 0:
    print(f"   이상치 자치구:")
    display(crime_outliers[['자치구', 'CCTV효과범죄_합계']])

# 인구 이상치 탐지
print("\n🔍 인구 이상치 탐지 (IQR 방법):\n")
pop_outliers, pop_lower, pop_upper = detect_outliers_iqr(population_df, '인구수')
print(f"   정상 범위: {pop_lower:.0f} ~ {pop_upper:.0f}")
print(f"   이상치 개수: {len(pop_outliers)}개")
if len(pop_outliers) > 0:
    print(f"   이상치 자치구:")
    display(pop_outliers[['자치구', '인구수']])

In [ ]:
# 이상치 처리 방안 결정
outlier_decision = """
╔═══════════════════════════════════════════════════════════════╗
║             이상치 처리 방안 결정                              ║
╚═══════════════════════════════════════════════════════════════╝

📋 탐지된 이상치:
   - CCTV: {cctv_outliers}개 자치구
   - 범죄: {crime_outliers}개 자치구
   - 인구: {pop_outliers}개 자치구

🎯 처리 방안:
   ✅ 이상치 유지 (제거하지 않음)

💡 근거:
   1. 샘플 데이터이므로 실제 데이터와 다를 수 있음
   2. 실제 서울시에서도 자치구별 편차가 클 수 있음
      (예: 강남구 vs 중구, 유동인구 차이)
   3. 이상치가 실제 현상을 반영할 가능성 있음
   4. 분석 과정에서 인구당 비율로 표준화하므로 영향 최소화

⚠️ 주의사항:
   - Day 3에서 파생변수 생성 시 이상치 영향 재확인
   - 회귀분석 시 Cook's Distance로 영향력 재평가
   - 필요 시 Robust 회귀 기법 고려

""".format(
    cctv_outliers=len(cctv_outliers),
    crime_outliers=len(crime_outliers),
    pop_outliers=len(pop_outliers)
)

print(outlier_decision)

# 처리 방안을 파일로 저장
outlier_path = os.path.join(DATA_PATHS['reports'], 'day2_outlier_decision.txt')
os.makedirs(os.path.dirname(outlier_path), exist_ok=True)
with open(outlier_path, 'w', encoding='utf-8') as f:
    f.write(outlier_decision)
print(f"\n✅ 이상치 처리 방안 저장: {outlier_path}")

## 7. 정제된 데이터 저장

In [ ]:
# 정제된 데이터 저장 (processed 폴더에)
print("💾 정제된 데이터 저장 중...\n")

# 저장 경로
cctv_clean_path = os.path.join(DATA_PATHS['processed'], 'cctv_cleaned.csv')
crime_clean_path = os.path.join(DATA_PATHS['processed'], 'crime_cleaned.csv')
pop_clean_path = os.path.join(DATA_PATHS['processed'], 'population_cleaned.csv')

# 저장
save_csv_safely(cctv_df, cctv_clean_path, index=False, encoding='utf-8-sig')
save_csv_safely(crime_df, crime_clean_path, index=False, encoding='utf-8-sig')
save_csv_safely(population_df, pop_clean_path, index=False, encoding='utf-8-sig')

print("\n✅ 모든 정제된 데이터 저장 완료")

In [ ]:
# 정제 후 데이터 크기 확인
print("\n📊 정제 후 데이터 크기:")
print(f"   CCTV: {cctv_df.shape[0]}행 × {cctv_df.shape[1]}열")
print(f"   범죄: {crime_df.shape[0]}행 × {crime_df.shape[1]}열")
print(f"   인구: {population_df.shape[0]}행 × {population_df.shape[1]}열")

# 컬럼 목록 확인
print("\n📋 정제 후 컬럼 목록:")
print(f"\nCCTV ({len(cctv_df.columns)}개):")
print(list(cctv_df.columns))
print(f"\n범죄 ({len(crime_df.columns)}개):")
print(list(crime_df.columns))
print(f"\n인구 ({len(population_df.columns)}개):")
print(list(population_df.columns))

## 8. Day 2 요약 및 다음 단계

In [ ]:
# Day 2 요약 리포트
summary = f"""
{'='*70}
Day 2 데이터 정제 및 구조 파악 요약
{'='*70}

1. 데이터 로드 및 표준화
   ✅ Day 1에서 생성한 샘플 데이터 로드 성공
   ✅ 자치구명 표준화 완료
   ✅ 자치구명 일치 여부 확인: {'일치' if consistency_result['consistent'] else '불일치'}

2. CCTV 데이터 분석
   - 평균 총 CCTV: {cctv_df['총_CCTV'].mean():.0f}대
   - 방범용 비율: {cctv_df['방범용_비율'].mean():.2f}%
   - 교통단속용 비율: {cctv_df['교통단속용_비율'].mean():.2f}%
   - 어린이안전용 비율: {cctv_df['어린이안전용_비율'].mean():.2f}%
   - CCTV 최다 설치 자치구: {cctv_df.loc[cctv_df['총_CCTV'].idxmax(), '자치구']} ({cctv_df['총_CCTV'].max()}대)
   - CCTV 최소 설치 자치구: {cctv_df.loc[cctv_df['총_CCTV'].idxmin(), '자치구']} ({cctv_df['총_CCTV'].min()}대)

3. 범죄 데이터 분석
   - 평균 총 범죄: {crime_df['총_범죄'].mean():.0f}건
   - CCTV효과범죄 비율: {crime_df['CCTV효과범죄_비율'].mean():.2f}%
   - 절도 평균: {crime_df['절도'].mean():.0f}건
   - 강도 평균: {crime_df['강도'].mean():.0f}건
   - 범죄 최다 발생 자치구: {crime_df.loc[crime_df['CCTV효과범죄_합계'].idxmax(), '자치구']} ({crime_df['CCTV효과범죄_합계'].max()}건)
   - 범죄 최소 발생 자치구: {crime_df.loc[crime_df['CCTV효과범죄_합계'].idxmin(), '자치구']} ({crime_df['CCTV효과범죄_합계'].min()}건)

4. 인구 데이터 분석
   - 평균 인구: {population_df['인구수'].mean():.0f}명
   - 평균 인구밀도: {population_df['인구밀도'].mean():.0f}명/km²

5. 이상치 탐지 결과
   - CCTV 이상치: {len(cctv_outliers)}개 자치구
   - 범죄 이상치: {len(crime_outliers)}개 자치구
   - 인구 이상치: {len(pop_outliers)}개 자치구
   - 처리 방안: 이상치 유지 (실제 현상 반영 가능성)

6. 정제된 데이터 저장
   ✅ CCTV: {cctv_df.shape[0]}행 × {cctv_df.shape[1]}열
   ✅ 범죄: {crime_df.shape[0]}행 × {crime_df.shape[1]}열
   ✅ 인구: {population_df.shape[0]}행 × {population_df.shape[1]}열

7. 다음 단계 (Day 3 - 2025-07-06)
   - 세 개 데이터프레임 병합 (자치구 기준)
   - 파생변수 생성:
     * 인구당 CCTV (총_CCTV / 인구수 × 1000)
     * 인구당 방범CCTV (방범용 / 인구수 × 1000)
     * 인구당 범죄율 (CCTV효과범죄 / 인구수 × 1000)
     * 절도율, 강도율, 차량범죄율 등
   - 통합 데이터 저장 및 검증

{'='*70}
"""

print(summary)

# 요약 파일 저장
summary_path = os.path.join(DATA_PATHS['reports'], 'day2_summary.txt')
with open(summary_path, 'w', encoding='utf-8') as f:
    f.write(summary)
print(f"✅ 요약 파일 저장: {summary_path}")

print("\n" + "="*70)
print("🎉 Day 2 작업 완료!")
print("="*70)